## 实验：模型超参数调优

### 实验概述

模型优化是机器学习算法实现中最困难的挑战之一。机器学习和深度学习理论的所有分支都致力于模型的优化。

机器学习中的超参数优化旨在寻找使得机器学习算法在验证数据集上表现性能最佳的超参数。超参数与一般模型参数不同，超参数是在训练前提前设置的。

### 超参数优化方法

超参数的设置对于模型性能有着直接影响，其重要性不言而喻。为了最大化模型性能，了解如何优化超参数至关重要。接下来介绍了几种常用的超参数优化方法。

**1. 手动调参**

很多情况下，工程师们依靠试错法手动对超参数进行调参优化，有经验的工程师能够很大程度上判断超参数如何进行设置能够获得更高的模型准确性。但是，这一方法依赖大量的经验，并且比较耗时，因此发展出了许多自动化超参数优化方法。

**2. 网格化寻优（Grid Search）**

网格化寻优可以说是最基本的超参数优化方法。使用这种技术，我们只需为所有超参数的可能构建独立的模型，评估每个模型的性能，并选择产生最佳结果的模型和超参数。

<img src="./img/grid.jpg" width="40%">

**3. 随机寻优（Random Search）**

通常并不是所有的超参数都有同样的重要性，某些超参数可能作用更显著。 而随机寻优方法相对于网格化寻优方法能够更准确地确定某些重要的超参数的最佳值。

<img src="./img/random.jpg" width="40%">

随机寻优方法在超参数网格的基础上选择随机的组合来进行模型训练。 可以控制组合的数量，基于时间和计算资源的情况，选择合理的计算次数。 这一方法可以通过调用 Scikit-Learn 库中的 `randomizedSearchCV` 函数来实现。

**4. 基于梯度的优化方法（Gradient-based Optimization）**

基于梯度的优化方法经常被用于神经网络模型中，主要计算超参数的梯度，并且通过梯度下降算法进行优化。这一方法的应用场景并不广泛，其局限性主要在于：

<img src="./img/gradient.jpg" width="40%">

1. 超参数优化通常不是一个平滑的过程
2. 超参数优化往往具有非凸的性质

**5. 进化寻优（Evolutionary Optimization）**

进化寻优方法的思想来源于生物学概念，由于自然进化是不断变化的环境中发生的一个动态过程，因此适用于超参数寻优问题，因为超参数寻优也是一个动态过程。

<img src="./img/evolutionary.jpg" width="40%">

进化算法经常被用来寻找其他技术不易求解的近似解。优化问题往往没有一个精确的解决方案，因为它可能太耗时并且计算资源占用很大。在这种情况下，进化算法通常可以用来寻找一个足够的近似最优解。 进化算法的一个优点是，它们可以产生出不受人类误解或偏见影响的解决方案。

作为一个一般性的经验法则，任何时候想要优化调整超参数，优先考虑网格化寻优方法和随机寻优方法！

### 实验目标

本次实验将使用网格搜索对神经网络的超参数进行调优，寻找最优的网络模型。

### 1. 导入库

首先，让我们导入一些模块，确保 `MatplotLib` 内联绘制图形，并准备一个函数来保存图形。我们还检查是否安装了 `Python 3.5` 或更高版本，以及 `Scikit-Learn≥0.20`。

In [1]:
# 要求Python ≥3.5 
import sys
assert sys.version_info >= (3, 5)

# 要求Scikit-Learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

import pandas as pd
import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. 定义路径

定义数据文件路径以及存储路径。

In [2]:
base_path = os.environ.get("BASE_PATH",'../data/')
data_path = os.path.join(base_path + "lab12/") 
result_path = "result"
img_path = "img"

os.makedirs(result_path, exist_ok=True)

### 3. 定义存储函数

定义存储函数用来存储生成的图片数据。

In [3]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(result_path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### 4. 数据加载和处理

### 4.1 加载数据集

利用 `fetch_california_housing` 加载内置数据集。

In [4]:
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from tensorflow import keras

housing = fetch_california_housing(data_home=data_path)

### 4.2 数据切分

将加载的数据集切分为训练集，验证集和测试集。

In [5]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, 
                                                              housing.target, 
                                                              random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, 
                                                      y_train_full, random_state=42)

### 4.3 数据标准化

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### 5. 超参数调优

### 5.1 设置随机种子

In [7]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

### 5.2 定义模型构建函数

In [8]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    # 输入层
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    # 构建多层隐藏层
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    # 输出层
    model.add(keras.layers.Dense(1))
    # 优化器
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    # 模型编译
    model.compile(loss="mse", optimizer=optimizer)
    return model

### 5.3 实现 Scikit-Learn 回归器接口

In [9]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

### 5.4 模型训练

In [10]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 4s 8ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 3s 7ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 2s 6ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4060 - val_l

### 5.5 计算得分

In [11]:
mse_test = keras_reg.score(X_test, y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.3412


### 5.6 模型预测

In [12]:
y_pred = keras_reg.predict(X_test[:3])
y_pred

array([0.6514431, 1.6107498, 4.071351 ], dtype=float32)

### 5.7 网格搜索

使用网格搜索对超参数进行调优。

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)               .tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2)      .rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=4, n_hidden=1, learning_rate=0.022174573948353458 .....
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


242/242 [==============================] - 2s 7ms/step - loss: 0.8420 - val_loss: 0.4703
Epoch 2/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4815 - val_loss: 0.4247
Epoch 3/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4519 - val_loss: 0.4052
Epoch 4/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4429 - val_loss: 0.3975
Epoch 5/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4368 - val_loss: 0.3991
Epoch 6/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4340 - val_loss: 0.4031
Epoch 7/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4351 - val_loss: 0.4043
Epoch 8/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4267 - val_loss: 0.3929
Epoch 9/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4258 - val_loss: 0.4040
Epoch 10/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4237 - val_loss: 0.3886
E

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.6s remaining:    0.0s


242/242 [==============================] - 2s 5ms/step - loss: 0.7452 - val_loss: 0.4860
Epoch 2/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4649 - val_loss: 0.4280
Epoch 3/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4495 - val_loss: 0.5791
Epoch 4/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4438 - val_loss: 0.4549
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4414 - val_loss: 0.5250
Epoch 6/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4409 - val_loss: 0.5486
Epoch 7/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4388 - val_loss: 0.5871
Epoch 8/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4381 - val_loss: 0.4759
Epoch 9/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4371 - val_loss: 0.7523
Epoch 10/100
242/242 [==============================] - 1s 4ms/step - loss: 0.4369 - val_loss: 0.7478
E

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  9.9min finished


363/363 [==============================] - 1s 2ms/step - loss: 0.8194 - val_loss: 1.8036
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4857 - val_loss: 2.0827
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4266 - val_loss: 0.3796
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3857 - val_loss: 0.4283
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3740 - val_loss: 0.3617
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3620 - val_loss: 0.4566
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3566 - val_loss: 0.3573
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3492 - val_loss: 0.3380
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3432 - val_loss: 0.3757
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3389 - val_loss: 0.4069
E

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f7fd0cf6240>,
                   param_distributions={'learning_rate': [0.001683454924600351,
                                                          0.02390836445593178,
                                                          0.008731907739399206,
                                                          0.004725396149933917,
                                                          0.0006154014789262348,
                                                          0.0006153331256530192,
                                                          0.0003920021771415983,
                                                          0.01619845322936229,
                                                          0.004779156784872302,
                                                          0.0...
                                                          0.00502142573

### 5.8 输出最优参数

In [15]:
rnd_search_cv.best_params_

{'n_neurons': 74, 'n_hidden': 3, 'learning_rate': 0.005803602934201024}

### 5.9 输出最优得分

In [16]:
rnd_search_cv.best_score_

-0.31833014885584515

### 5.10 选择最优模型

In [17]:
rnd_search_cv.score(X_test, y_test)

model = rnd_search_cv.best_estimator_.model
model

162/162 [==============================] - 0s 1ms/step - loss: 0.3029


### 5.11 最优模型评估

In [18]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.3029


0.30288204550743103

### 实验小结

本实验主要使用网格搜索的方法对神经网络进行超参数调优，从而得到了最优模型。